In [ ]:
%use lets-plot

In [ ]:
@file:DependsOn("com.github.rbrott:road-runner2:+")

In [ ]:
import com.acmerobotics.roadrunner2.*


typealias PoseHistory = MutableList<Transform2<World, *, DoubleNum>>


val path = TangentPath(
    ArcApproxArcCurve2(
        QuinticSpline2(
            QuinticSpline1(0.0, 20.0, 0.0, 30.0, 30.0, 0.0, 4),
            QuinticSpline1(0.0, 20.0, 0.0, 15.0, 10.0, 0.0, 4)
        )
    )
)

val maxVel = 1.0
val profile = Profile(
    listOf(
        Profile.ConstVelSegment(
            DualNum.constant(0.0, 4),
            maxVel, path.length / maxVel
        )
    )
)
val trajectory = Trajectory(path, profile)

tailrec fun <Robot> step(pose: Transform2<World, Robot, DualNum<Time>>, lastProject: Double,
                 targets: PoseHistory, measured: PoseHistory): Pair<PoseHistory, PoseHistory> {
    val s = trajectory.project<Robot>(Position2.bind(pose.translation), lastProject)

    if (s.value() >= path.length - 0.25) {
        return Pair(targets, measured)
    }

    val targetPose = trajectory.getByDisp<Robot>(s.value())

    measured.add(pose.constant())
    targets.add(targetPose.constant())

    val error = targetPose.constant() - pose.constant()
    val correction = Twist2<DualNum<Time>>(
        DualNum(listOf(0.0, 10.0 * error.x.value)),
        DualNum(listOf(0.0, 10.0 * error.y.value)),
        DualNum(listOf(0.0, 0.01 * error.theta.value))
    )

    class CommandRobot
    val command = targetPose.plus<CommandRobot>(correction)

    val dt = 0.01
    val nextPose = pose.plus<Robot>(
        Twist2(
            command.translation.x.drop(1) * dt,
            command.translation.y.drop(1) * dt,
            command.rotation.log().drop(1) * dt
        )
    )

    return step(nextPose, s.value(), targets, measured)
}

class InitialTraj
val initialPose = trajectory.get<InitialTraj>(0.0)

class InitialRobot
val perturbation =
    Transform2.rotateThenTranslate<InitialTraj, InitialRobot, DualNum<Time>>(
        Rotation2(DualNum.constant(cos(PI / 8.0), 10), DualNum.constant(sin(PI / 8.0), 10)),
        Vector2(DualNum.constant(1.0, 10), DualNum.constant(-3.0, 10))
    )
val perturbedPose = initialPose * perturbation

val (targets, measured) = step(perturbedPose, 0.0, mutableListOf(), mutableListOf())

In [ ]:
val data = mapOf<String, List<*>>(
    "pose" to targets.map { "target" } + measured.map { "measured" },
    "x" to targets.map { it.translation.x.value } + measured.map { it.translation.x.value },
    "y" to targets.map { it.translation.y.value } + measured.map { it.translation.y.value }
)

In [ ]:
letsPlot(data) { x = "x"; y = "y"; color = "pose" } + geomPoint(shape = 1)